# Final Project - Connect 4

Joseph Larson, Joel Kraft, Spencer Lofing

In [12]:
import copy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

state = [['R', 'B', 'R', 'B', 'R', 'B'], [], ['B'], [], [], [], []]

def printState(state):
    copystate = copy.deepcopy(state)
    for slot in copystate:
        slotlength = len(slot)
        for i in range(0, 6 - slotlength):
            slot.insert(0, 'X')

    for i in range(0, 6):
        for slot in copystate:
            print(slot[i] + ' '),
        print()

def validMoves(state):
    listOfMoves = []
    for i in range(0, len(state)):
        if len(state[i]) < 6:
            listOfMoves.append(i)
    return listOfMoves

def makeMove(state, move, color):
    state[move].insert(0, color)

In [15]:
def stateMoveTuple(state, move, color) :
    return ((tuple(state[0]), tuple(state[1]), tuple(state[2]), tuple(state[3]), tuple(state[4]), tuple(state[5]), tuple(state[6])), move, color)

In [16]:
stateMoveTuple(state, 3, 'R')

((('R', 'B', 'R', 'B', 'R', 'B'), (), ('B',), (), (), (), ()), 3, 'R')

In [8]:
validMoves(state)

[1, 2, 3, 4, 5, 6]

In [9]:
makeMove(state, 1, "R")

In [10]:
state

[['R', 'B', 'R', 'B', 'R', 'B'], ['R'], ['B'], [], [], [], []]

In [11]:
def trainQ(nRepetitions, learningRate, epsilonDecayFactor, validMovesF, makeMoveF, startState = [[], [], [], [], [], [], []]) :
    epsilon = 1.0
    Q = {}
    steps = []
    def puzzleSolved(state) : 
        return len(state[0]) == 0 and len(state[1]) == 0 and len(state[2]) != 0

    ## TODO: replace with the actual function for telling if the game has been won
    
    for repetition in range(nRepetitions):
        epsilon *= epsilonDecayFactor
        step = 0
        state =  startState
        done = False
        while not done:
            step += 1
            move = epsilonGreedy(epsilon, Q, state, validMovesF)
            stateNew = makeMove(state, move)
            if stateMoveTuple(state,move) not in Q:
                Q[stateMoveTuple(state,move)] = -1  # initial Q value for new board,move

            if puzzleSolved(stateNew):
                # solved!
                Q[stateMoveTuple(state,move)] = 0
                done = True
                
            if step > 1:
                Q[stateMoveTuple(stateOld,moveOld)] += learningRate * (-1 + Q[stateMoveTuple(state,move)] - Q[stateMoveTuple(stateOld,moveOld)])

            stateOld, moveOld = state, move # remember board and move to Q(board,move) can be updated after next steps
            state = stateNew
        steps.append(step)
    return Q, np.array(steps)

In [13]:
def epsilonGreedy(epsilon, Q, state, validMovesF):
    validMoves = validMovesF(state)
    if np.random.uniform() < epsilon:
        # Random Move
        return validMoves[np.random.choice(len(validMoves))]
    else:
        # Greedy Move
        Qs = np.array([Q.get(stateMoveTuple(state, m), 0) for m in validMoves]) 
        return validMoves[ np.argmax(Qs) ]

In [14]:
def testQ(Q, maxSteps, validMovesF, makeMoveF, startState = [[], [], [], [], [], [], []]) : 
    state = startState
    listOfStates = []
    def puzzleSolved(state) : 
        return len(state[0]) == 0 and len(state[1]) == 0 and len(state[2]) != 0
    
    ## TODO: replace puzzleSolved with the actual function

    for steps in range(maxSteps) : 
        listOfStates.append(state)
        move = epsilonGreedy(0, Q, state, validMovesF)
        stateNew = makeMove(state, move, 'R')
        state = stateNew
        if puzzleSolved(stateNew) :
            listOfStates.append(stateNew)
            return listOfStates
    return []